In [1]:
import os, sys
import matplotlib.pyplot as plt
import numpy as np
from sklearn import decomposition, manifold
import h5py

% matplotlib notebook

In [2]:
def compute_distance(x,y):
    x = x / np.linalg.norm(x)
    y = y / np.linalg.norm(y)
    return np.linalg.norm(x-y)

In [3]:
def print_percentage(n, t):
    sys.stdout.write('\r')
    sys.stdout.write("[%-20s] %d%%" % ('=' * ((n * 20/t) + 1) , n * 100/t + 1 ))
    sys.stdout.flush()

In [74]:
np.array(feature_map).shape

(720, 4096)

In [79]:
patsfile = h5py.File('/fileserver/nmec-handwriting/localfeatures/iam_fiel_connected_component_activations_nmec.hdf5','r')

feature_map = []
for i, feats in enumerate(patsfile):
    # meanfeats = np.expand_dims( patsfile[feats].value.mean(axis=0), 0 )
    meanfeats = np.array(patsfile[feats].value.mean(axis=0), dtype='float')
    if meanfeats.shape==():
        meanfeats = np.array(np.ones(4096))
    feature_map = feature_map + [meanfeats]
    print_percentage(i, len(patsfile.keys()))
        
feature_map = np.array(feature_map)
feature_map.shape

files = patsfile.keys()

[====================] 100%

In [33]:
pca = decomposition.PCA(n_components=400)
transform = pca.fit_transform(feature_map)
# transform = transform.T
transform.shape

(358, 10)

In [ ]:
a = transform[0]
b = transform[1]
compute_min_distance(a,b)

In [76]:
transform = feature_map

In [77]:
metric = []
for i, image in enumerate(transform):
    metricline = [np.array([compute_distance(image, other) for other in transform])]
    metric += metricline
    print_percentage(i, len(transform))

metric = np.array(metric)
F = -metric
np.fill_diagonal(F, -sys.maxint)

[====================] 100%

In [80]:
soft_correct = 0
hard_correct = 0
total_num = 0

k = 10
g = 8
max_top = 3

for j, i in enumerate(F):
    
    total_num += 1
    topk = i.argsort()[-k:]
    
    if files[j][:6] in (files[index][:6] for index in topk):
        soft_correct += 1
    
    hardsample = list(files[index][3:6] for index in topk[-max_top:])
    if len(set(hardsample)) == 1 and hardsample[0] == files[j][3:6]:
        print "%s matched %s" % (files[j][3:10], hardsample)
        hard_correct += 1

print "%-30s" % ( "-" * 37 )
print "SOFT CRITERIA: Top %d\t= %f" %(k, (soft_correct + 0.0) / total_num)
print "HARD CRITERIA: Top %d\t= %f" %(max_top, (hard_correct + 0.0) / total_num)

003-005 matched [u'003', u'003', u'003']
006-002 matched [u'006', u'006', u'006']
006-004 matched [u'006', u'006', u'006']
008-005 matched [u'008', u'008', u'008']
009-005 matched [u'009', u'009', u'009']
015-005 matched [u'015', u'015', u'015']
015-006 matched [u'015', u'015', u'015']
026-001 matched [u'026', u'026', u'026']
026-003 matched [u'026', u'026', u'026']
026-004 matched [u'026', u'026', u'026']
038-005 matched [u'038', u'038', u'038']
056-001 matched [u'056', u'056', u'056']
056-003 matched [u'056', u'056', u'056']
056-005 matched [u'056', u'056', u'056']
056-006 matched [u'056', u'056', u'056']
060-001 matched [u'060', u'060', u'060']
060-002 matched [u'060', u'060', u'060']
060-005 matched [u'060', u'060', u'060']
060-006 matched [u'060', u'060', u'060']
060-008 matched [u'060', u'060', u'060']
-------------------------------------
SOFT CRITERIA: Top 10	= 0.461111
HARD CRITERIA: Top 3	= 0.027778
